In [1]:
# inline will cause problem when closing animation plots
%precision 16
import numpy
import matplotlib.pyplot as plt
import pandas as pd

# Final Project
- by Derek Che (yc4608) & Thomas Wang (tw3003) for APMA 4300 Introduction to Numerical Methods

## Problem Description [15 pts]

As the computing power grows in the past century, it becomes more viable for computational physicists to perform detailed and precise simulation of matter's interaction with fewer approximation here and there. However, single-particle simulation remains to be one of the most formidable barrier since the amount of resources required can be hardly found in common places outside of supercomputer sites and national laboratories. Even for the case of collisionless and incompressible fluid-flow motion of identical particles--a relatively simple physical interaction between matters--this problem still exists, provoking scientists to look for simplifications in modeling that permits acceptable accuracy and time complexity.

Looking back at the simplified models proposed by many researchers, it's noticeable that most of them treat such a particle flow as a continuous medium rather than discrete particles. Thus, the **Navier-Stokes** equations, a physical relation for viscous fluid derived from physics laws like momentum balance of Newtonian fluids and conservation of mass, become an ideal description of fluid in Computational Fluid Dynamics:

$$
\nabla \cdot \mathbf{u} = 0 \tag{1}
$$

$$
\frac{\partial \mathbf{u}}{\partial t} = 
-(\mathbf{u} \cdot \nabla)\mathbf{u} 
-\frac{1}{\rho} \nabla p 
+ \nu \nabla^2 \mathbf{u} 
+ \mathbf{f}\tag{2}
$$

where $u$ is the velocity field, $p$ is the pressure field, $\nu$ is the kinematic viscosity, $\rho$ is the fluid's density, and $f$ is an external force. Nevertheless, it's not so simple to solve the Navier-Stokes equations: there are many numerical schemes with different level of accuracy and stability that solve equation, and understanding what our solution will be used for is essential for making the appropriate decision. 

For us, the accuracy of physical quantities of the fluid is not of our primary concern; enthusiastic about realistic animation of matter flows, we are more interested in the intricate shape and movement of the particles forming a viscous fluid--the same goal as Jos Stam, who was a researcher at a 3D graphics software company called Alias|wavefront in 1999. In his famous paper titled [*Stable Fluids*](https://dl.acm.org/doi/pdf/10.1145/311535.311548), he proposed an unconditionally stable solver of Navier-Stokes equations where stability is not influenced by the length of time step. His numerical scheme is also known for its relative simplicity in interpretation and implementation, so we chose to reproduce his solver in 2D space and by enforcing periodic boundary conditions. 

The reason we assume periodic boundary conditions is that, when solving the Navier-Stokes equations, we can represent the functions as linear combinations of sinusoidal waves. We can then use Fast Fourier Transform (FFT) to significantly reduce implementation difficulty as many linear operations like the gradient or the Laplacian that are used in Stam's solver become diagonal operators in the Fourier domain. Instead of carrying out complicated computations in the spatial domain, we can transform to the Fourier domain, perform the much easier computations, and transform back. Details on the solver and why FFT is an elegant way to implement the solver are provided in the **Computational Methods** part below. Apart from making our calculations easier, FFT is a cornerstone of modern computational mathematics and engineering, enabling efficient analysis of data across numerous fields such as signal processing, medical imaging, physics simulations, financial modeling, and machine learning. Therefore, we thought understanding and implementing the FFT algorithm itself would be an interesting and meaningful process. 

Therefore, for the final project, We will implement from scratch the solver and the accompanying Fast Fourier Transform package, and we will visualize the results as a flow of viscous fluid and its constituent particles. For simplicity, we will not implement the evolution of other scalar quantity moving with the fluid and the real-time user interaction. Instead, when running the solver, we will impose an initial force that applies to the evolution for only at its start (e.g., the first second), after which the fluid is left to evolve by itself.

**Note**:

This is a collaborative final project between Derek Che and Thomas Wang. We collaborated closely on the selection of topics and planning of the actual implementations. Together, we discussed the direction of the project, deciding what functions to implement, why they were needed, and how to implement them at a high level.

In terms of actual coding, Derek Che was responsible for the implementation of the DFT-FFT package and its testing, while Thomas Wang was responsible for the implementation of the solver code. We also collaborated on the text portion of the project, with Derek Che explaining everything related to FFT and Thomas Wang focusing on the solver.

We worked closely to merge our contributions, ensuring the notebook flows cohesively. Additionally, we collaborated on writing the problem distribution discussions and other shared sections of the notebook to ensure clarity and coherence.



## Problem Justification [5 pts] 

The problem of visualizing matter flow without resorting to the single-particle simulation is common among visual effect designers looking for animation of intricate turbulences of fluids like smoke and cloud. They are not into the physics of the fluids, so trading physical accuracy for better time complexity and more detailed fluid motion works the best for them. Jos Stam's method also permits long time steps, making efficient simulation of long-term movement of fluids possible for animators. For us, this satisfies our curiosity in the creation of visual effect for fluids because we were fascinated by simulation animation of fluids in the context of race car aerodynamics, and we are excited to implement this and generate our own fluid flow.

## Computational  Methods [10 pts]



#### Fast Fourier Transform
One important property that we will exploit is that many linear operations like the gradient or the Laplacian becomes diagonal operator. Here is the proof: 

Under periodic boundary conditions, a function $\mathbf{u}(\mathbf{x})$ can be expressed as 
$$
\mathbf{u}(\mathbf{x}) = \sum_\mathbf{k} \hat{\mathbf{u}}(\mathbf{k})e^{i \mathbf{k} \cdot \mathbf{x}}, \quad \mathbf{k} = \frac{2 \pi}{L}\mathbf{n}
$$
where $\hat{\mathbf{u}}(\mathbf{k})$ is the Fourier coefficients.
1. For gradient operator $\nabla$:
    $$
    \begin{aligned}
    \nabla \mathbf{u}(\mathbf{x}) &= \left(\frac{\partial \mathbf{u}}{\partial x}, \frac{\partial \mathbf{u}}{\partial y}, \frac{\partial \mathbf{u}}{\partial z}\right) \\
    &= \nabla \left(\sum_\mathbf{k} \hat{\mathbf{u}}(\mathbf{k})e^{i \mathbf{k} \cdot \mathbf{x}} \right) \\
    &= \sum_\mathbf{k} \hat{\mathbf{u}}(\mathbf{k})\nabla e^{i \mathbf{k} \cdot \mathbf{x}}\\
    &= \sum_\mathbf{k} \hat{\mathbf{u}}(\mathbf{k})\left(i\mathbf{k}\right) e^{i \mathbf{k} \cdot \mathbf{x}} = \sum_\mathbf{k} i\mathbf{k}\hat{\mathbf{u}}(\mathbf{k}) e^{i \mathbf{k} \cdot \mathbf{x}} 
    \end{aligned}
    $$
    Taking the Fourier transform on both sides, we obtain:
    $$
    \mathcal{F}\{(\nabla\mathbf{u}(\mathbf{x}))\} = i\mathbf{k}\hat{\mathbf{u}}(\mathbf{k})
    $$
2. For Laplacian Operator, we can start from the gradient result
    $$
    \begin{aligned}
    \nabla \cdot (\nabla \mathbf{u}(\mathbf{x})) &= \nabla^2 \mathbf{u}(\mathbf{x}) \\
    \mathcal{F} \{\nabla^2 \mathbf{u}(\mathbf{x})\} &= \mathcal{F} \{\nabla \cdot (\nabla \mathbf{u}(\mathbf{x}))\} \\
    &= \nabla \cdot \mathcal{F} \{\nabla \mathbf{u}(\mathbf{x})\} \\
    &= \nabla \cdot (i\mathbf{k}\hat{\mathbf{u}}(\mathbf{k})) \\
    &= i\mathbf{k} \cdot (i\mathbf{k}\hat{\mathbf{u}}(\mathbf{k})) \\
    &= -k^2\hat{\mathbf{u}}(\mathbf{k})
    \end{aligned}
    $$
    As a result,
    $$
    \mathcal{F} \{\nabla^2 \mathbf{u}(\mathbf{x})\} = -k^2\hat{\mathbf{u}}(\mathbf{k})
    $$
Using these results, the diffusion and projection step of our solver as explained below can be calculated more elegantly by first transforming to the Fourier domain. 

#### Solver Details

Following Jos Stam's paper, we introduce a method that, given $\mathbf{u(x}, t)$, evolves $\mathbf{u(x,} t+\Delta t)$ by applying the RHS of Navier-Stokes in sequence. The solver itself is not limited to periodic boundary conditions, but since PBCs are our focus, we interweave the FFT and IFFT steps in between the main steps of the solver and show how they speed up the calculations. 

To obtain a inductive relation solely of $\mathbf{u}$, let us try to eliminate the $p$ in equation(2) with information from equation(1). Notice a mathematical property that any vector field $w$ can uniquely be decomposed into the form
$$
\mathbf{w = u} + \nabla q
$$
where $u$ has zero divergence and q is a scalar field. This allows us to defined $\mathbf{P = w} - \nabla q$ that projects a vector to its divergence-free part $\mathbf{u = Pw}$. Notice that $\mathbf{P}$ is implicitly defined when both sides are taken their divergence:
$$
\nabla \cdot \mathbf{w} = \nabla^2 q
$$
Applying $\mathbf{P}$ on both side and noticing that $\mathbf{Pu = u}$ and $\mathbf{P} \nabla p = 0$, we arrive at the single equation to according to which $u$ is evolved:
$$
\frac{\partial \mathbf{u}}{\partial t} = \mathbf{P} (
-(\mathbf{u} \cdot \nabla)\mathbf{u} 
+ \nu \nabla^2 \mathbf{u} 
+ \mathbf{f})\tag{3}
$$

Now we can apply operator splitting method where we will resolve each term on the RHS step by step as if they act independently within a time step. This step ensures high stability at the expense of physical accuracy. 

**Add Force**

Let $\mathbf{u(x},t) = \mathbf{w_{0}}$. 
The first step is adding a force, which is trivial:
$$
\mathbf{w_{1} = w_{0}} + \Delta t \mathbf{f(x,t)}
$$

**Advection**

This step deal with the $-(\mathbf{u} \cdot \nabla)\mathbf{u}$ term which introduces nonlinearity that bothered lots of scientists. Here, the *method of charcteristic* is applied, where the main goal is to look for the *characterstics* of the equation. Derivation is as follows:

If we are given the relationship of a scalar field $a\mathbf{(x},t)$ and a steady vector field $\mathbf{v(x},t)$ as follows:
$$
\frac{\partial a\mathbf{(x},t)}{\partial t} = 
-\mathbf{v(x}, t) \cdot \nabla a \mathbf{(x}, t) 
$$
and that at $t = 0$,  $a(\mathbf{x},0) = a_0(\mathbf{x})$ is known. We can let $\mathbf{p}(\mathbf{x_0},t)$ be the *characteristcs* that flow through $\mathbf{x_0}$ at $t=0$ and $\mathbf{x}$ at $t$:
$$
\frac{d\mathbf{p}\mathbf{(x_{0}},t)}{dt} = \mathbf{v(p(x_{0}},t)) \ \ \ \ \    and \ \ \ \ \  \mathbf{p(x_{0}},0) = \mathbf{x_{0}} \ \ \ \ \ and \ \ \ \ \ \mathbf{p(x_{0}},t) = \mathbf{x}
$$

Now, $a\mathbf{(x},t)$ can be written implicitly in $\mathbf{x}$ as $a(\mathbf{p(x_{0}},t),t)$, and notice an important relation:
$$
\frac{da\mathbf{(x},t)}{dt} = \frac{\partial a\mathbf{(x},t)}{\partial t} + \frac{\partial a\mathbf{(x},t)}{\partial \mathbf{p}} \ \frac{d\mathbf{p}\mathbf{(x_{0}},t)}{dt} = 0
$$
which indicates that the scalar field remains constant on the characteristics. Its value at $t$ can be traced back to any previous value on the characteristics, i.e.,
$$ 
a\mathbf{(x},t) = a\mathbf{(x_{0}},0) =  a_0\mathbf{(x_{0}})
$$
where $a_0$ is known.

For our case, $a$ is the components of $\mathbf{u}$, and $\mathbf{v} = \mathbf{u}$. This requires us to back track the particles from now to $- \Delta t$ and uses the $\mathbf{w_1}$ at that backtracked position as the current velocity $\mathbf{w_2}$:
$$
\mathbf{w_2(x) = w_1(p(x}, - \Delta t)) 
$$
Where $\mathbf{p(x}, - \Delta t) = \mathbf{p(x)} - \Delta t \ \mathbf{w_{1}}$. Since $\mathbf{w_{1}}$ might not have a value at the backtraced positions $\mathbf{p(x}, - \Delta t)$, a RegularGridInterpolator object with $\mathbf{w}_{1}$ as the input field is used to interpolate at $\mathbf{p(x}, - \Delta t)$.

**Fast Fourier Transform**

We can transform $\mathbf{w}(\mathbf{x})$ into the Fourier domain by 
$$
\hat{\mathbf{w}}_2 = \text{FFT}\{\mathbf{w}_2(\mathbf{x})\}
$$

**Diffusion**

To obtain better stability, Jos Stam proposed to rewrite the diffusion relation
$$
\frac{\partial \mathbf{w_2}}{\partial t} = \nu \nabla^2 \mathbf{w_2} 
$$
as an implicit relation
$$
(\mathbf{I}- \nu \Delta t\nabla^2 ) \ \mathbf{w_3(x)} = \mathbf{w_2(x)}
$$
where $\mathbf{I}$ is the identity operator. 

Since FFT is appllied to $\mathbf{w_2(x)}$ before the current step, we have its Fourier transform $\mathbf{\hat w_2(k)}$, and the relation is turned into
$$
(1 + \nu \Delta t k^2)\mathbf{\hat w_3(k)} = \mathbf{\hat w_2(k)}
$$

**Projection**

The projection step uses the implicit definition of the projection operator $\mathbf{P}$ to derive the scalar $q$, from which we can calculate the divergence-free part of $\mathbf{ w_3(x)}$:
$$
\nabla \cdot \mathbf{w_3} = \nabla^2 q\ \ \ \ \ \text{and} \ \ \ \ \ \mathbf{w_{4} = \mathbf{P}\ \mathbf{w_{3}} = w_{3}} - \nabla q
$$
In the Fourier domain, $q$ can be obtained much easier, and we arrive at
$$
\mathbf{P}\mathbf{w_{3}} \longrightarrow \widehat{\mathbf{P}}\hat{\mathbf{w}}_3(\mathbf{k}) = \hat{\mathbf{w_{3}}}(\mathbf{k}) - \frac{1}{k^2} \left( \mathbf{k} \cdot \hat{\mathbf{w_{3}}}(\mathbf{k}) \right)\mathbf{k}
$$
As a result,
$$
\hat{\mathbf{w}}_4(\mathbf{k}) = \widehat{\mathbf{P}}\hat{\mathbf{w}}_3(\mathbf{k})
$$

**Inverse Fast Fourier Transform**

We can then transform $\hat{\mathbf{w}}(\mathbf{k})$ back to the original domain by
$$
\mathbf{w}_4 = \text{FFT}^{-1}\{\hat{\mathbf{w}}_4(\mathbf{x})\}
$$


**Concolusion and Note**

The six steps combined is one step for our solver, and we can iterate for as many steps we want with arbitrary time step as the solver is unconditionally stable. 

Since the FFT is $O(n \log n)$, the version using FFT is slower than the version without FFT because Jos Stam proved in his paper that the solver is $O(n)$. Nevertheless, the much easier calulations and implementations provided by FFT lead us to pursue it.

In [18]:
# Provide complete installation or import information for external packages or modules here e.g.
import numpy
from scipy.interpolate import RegularGridInterpolator
import matplotlib
matplotlib.use('Qt5Agg') # for solver back end animation
import matplotlib.pyplot as plt

from numpy.testing import assert_allclose # for testing purposes

# from dft import freq, dft, idft, fft, ifft, fft2, ifft2
# our implementations of fft is also provided in the form a package

## Implementation [60 pts]

#### Fast Fourier Transform Implementation

We first implement the freq() function which will allow us to know what the computed Fourier coefficients correspond to.

In [22]:
def freq(n: int, d: float = 1.0) -> numpy.ndarray:
    """Compute the Discrete Fourier Transform frequencies.

    Parameters:
    -----------
        n (int): 
            The size of the FFT, which determines the number of frequency bins.
            Must be a positive integer. This corresponds to the number of samples 
            in the input data that will be used in FFT computation.
        d (float, optional): 
            The sampling interval between consecutive points in the input data.
            Default to 1.0, which corresponds to frequecies of cycles per sample.
            Change to other values when there is temporal meaning in data so that
            frequency can be interpreted appropriately (e.g. Hz).
            
    Returns:
    --------
        numpy.ndarray: 
            An array of frequency bins of length `n` that includes both positive and 
            negative frequencies. 
    
    Notes:
    ------
        - The first element corresponds to the DC component (0 Hz or 0 cycles/unit).
        - The positive frequencies are arranged in increasing order from index 0 to 
          `n//2`.
        - The negative frequencies are arranged in decreasing order from `-n//2` 
          to -1, following the Nyquist frequency if `n` is even.
    """
    coeff = 1.0 / (n * d)
    pos = numpy.arange(0, (n - 1) // 2 + 1, dtype=int)
    neg = numpy.arange(-(n // 2), 0, dtype=int)
    
    return numpy.concatenate((pos, neg)) * coeff

We test the function against the `numpy.fft.fftfreq()` function to show the correctness of our implementation.

In [24]:
def test_freq():
    # Test case 1: Small FFT size, default d
    n = 8
    d = 1.0
    expected = numpy.fft.fftfreq(n, d)
    result = freq(n, d)
    assert_allclose(result, expected, atol=1e-6)

    # Test case 2: Large FFT size, default d
    n = 1024
    d = 1.0
    expected = numpy.fft.fftfreq(n, d)
    result = freq(n, d)
    assert_allclose(result, expected, atol=1e-6)

    # Test case 3: Small FFT size, non-default d
    n = 8
    d = 0.1
    expected = numpy.fft.fftfreq(n, d)
    result = freq(n, d)
    assert_allclose(result, expected, atol=1e-6)

    # Test case 4: Large FFT size, non-default d
    n = 512
    d = 0.01
    expected = numpy.fft.fftfreq(n, d)
    result = freq(n, d)
    assert_allclose(result, expected, atol=1e-6)

    # Test case 5: Edge case, n=1
    n = 1
    d = 1.0
    expected = numpy.fft.fftfreq(n, d)
    result = freq(n, d)
    assert_allclose(result, expected, atol=1e-6)

    print("freq: All test cases passed!")

test_freq()

freq: All test cases passed!


Since FFT is just an efficent way to calculate the Discrete Fourier Transform (DFT), we also implement the naive DFT algorithm, which is based on matrix multiplication, hoping that later we can compare their performance and correctness.

In [26]:
def _dft(input: numpy.ndarray, axis: int, forward: bool) -> numpy.ndarray:
    """
    Compute the Discrete Fourier Transform (DFT) or its inverse along a specified axis.

    This is a helper function that calculates the DFT or inverse DFT based on the `forward` flag.
    It uses the matrix multiplication approach to perform the transform.

    Parameters:
    -----------
        input (numpy.ndarray): The input array to be transformed. Can be multi-dimensional.
        axis (int): The axis along which to compute the DFT. Default is 0.
        forward (bool): If True, computes the forward DFT. If False, computes the inverse DFT.
                        Default is True.

    Returns:
        numpy.ndarray: The transformed array with the same shape as the input.

    """
    # Move the specified axis to the second-to-last axis and expand a singleton dimension at the end
    # This is necessary because the @ operator aligns the last axis of the first matrix
    # with the second-to-last axis of the second matrix during matrix multiplication.
    f = numpy.moveaxis(input, axis, -1)[..., None]  

    N = f.shape[-2]

    # Construct the DFT matrix
    # For forward transform: exp(-2j * pi * k * n / N)
    # For inverse transform: exp(+2j * pi * k * n / N)
    DFT = numpy.exp(
        (-1)**forward * 2j * numpy.pi * numpy.arange(N) * numpy.arange(N)[..., None] / N
    )

    # Perform the matrix multiplication (DFT @ f) and remove the singleton dimension
    # Restore the original axis order after the computation
    result = numpy.moveaxis((DFT @ f)[..., -1], -1, axis)

    # Normalize by N for the inverse transform
    return result if forward else result / N


def dft(input: numpy.ndarray, axis: int = -1) -> numpy.ndarray:
    """
    Compute the Discrete Fourier Transform (DFT) along a specified axis, which
    converts data from its original domain to the frequency domain.

    Parameters:
    -----------
        input (numpy.ndarray): The input array to be transformed. Can be multi-dimensional.
        axis (int): The axis along which to compute the DFT. Default is -1.

    Returns:
    --------
        numpy.ndarray: The transformed array with the same shape as the input.
    """
    return _dft(input, axis, True)


def idft(input: numpy.ndarray, axis: int = -1) -> numpy.ndarray:
    """
    Compute the Inverse Discrete Fourier Transform (IDFT) along a specified axis, which
    converts data from frequency domain to the origal domain.

    Parameters:
    -----------
        input (numpy.ndarray): The input array to be transformed. Can be multi-dimensional.
        axis (int): The axis along which to compute the IDFT. Default is -1.

    Returns:
    --------
        numpy.ndarray: The transformed array with the same shape as the input.
    """
    return _dft(input, axis, False)

Now we implement the original Cooley-Tukey algorithm to compute FFT. One small change we make is that use a pre-computed DFT16 matrix and iteraitve bottum up approach instead of an recursive approach. However, the fundamental base-2, divide-and-conquer idea of Cooley-Tukey is unchanged. We will just switch back to naive DFT for input size smaller than 16, and use size 16 solutions and base-2 divide-and-conquer for larger input. In other words, we replace (size $2$ $\rightarrow$ size $4$ $\rightarrow$ $\cdots$ $\rightarrow$ size $\rightarrow$ $2^i$) with (size $16$ $\rightarrow$ size $32$ $\rightarrow$ $\cdots$ $\rightarrow$ size $2^i$) for $i > 16$ for better performance and less function call overheads. More details can be found in the docstring.

In [28]:
# pre-computed DFT matrix for length of 16
DFT_16 = numpy.exp(-2j * numpy.pi * numpy.arange(16) * numpy.arange(16)[:, None] / 16)

sizes = [1024 // (2 ** i) for i in range(5, -1, -1)] + [1024 * (2 ** i) for i in range(1, 9)]

# pre-computed twiddle factors dictionary
TWIDDLE_FACTOR = {
    N: numpy.exp(-2j * numpy.pi * numpy.arange(N // 2) / N)
    for N in sizes
}

def _fft(input: numpy.ndarray, axis: int, forward: bool) -> numpy.ndarray:
    """
    Compute the Fast Fourier Transform (FFT) using the Cooley-Tukey algorithm.

    This function implements an iterative version of the Cooley-Tukey FFT algorithm,
    which is efficient for input sizes that are powers of 2.

    Parameters:
    -----------
    input (numpy.ndarray): The input array to be transformed. Can be multi-dimensional.
    axis (int): The axis along which to compute the FFT. Default is -1.
    forward (bool): If True, computes the forward FFT. If False, computes the inverse DFT.
                    Default is True.

    Returns:
    --------
        numpy.ndarray: The transformed array with the same shape as the input.

    Raises:
    -------
    NotImplementedError
        - If the size of the input along the specified axis is not a power of 2.
        - If the size of the input is longer than 2^18 (pre-computed twiddle factor's upper limit)

    Notes:
    ------
        - The Cooley-Tukey Algorithm increases computing efficiency by exploiting symmetry of complex number.
          As an implementation of the original Cooley-Tukey Algorithm, this does means that only input size of
          power of 2 can be handled.
        - This implementation uses a base case of N_min = 16, below which it switches
          to a direct DFT computation.
        - The algorithm uses precomputed twiddle factors for efficiency.
        - For inverse FFT (forward=False), the output is scaled by 1/N.
    
    Main Loop Algorithm Explaination:
    ---------------------------------
        The Cooley-Tukey algorithm splits the computation of the DFT of size N
        into smaller DFTs of even-indexed and odd-indexed elements. This relies on 
        the following recursive relation:
        
            X[k] = DFT_N[k] = F_even[k] + W_N^k * F_odd[k]

            X[k + N/2] = F_even[k] - W_N^k * F_odd[k]
            
        Here:
        - F_even[k] is the DFT of the even-indexed elements of the input.
        - F_odd[k] is the DFT of the odd-indexed elements of the input.
        - W_N^k = exp(-2j * pi * k / N) is the twiddle factor for the size-N DFT.
        
        This recursion halves the problem size at each step, reducing the overall
        time complexity from O(N^2) (in naive DFT) to O(N log N).
        
        In this implementation:
        - F_even and F_odd correspond to the blocks in F[..., :F.shape[-1] // 2]
        and F[..., F.shape[-1] // 2:] respectively.
        - The twiddle factor is precomputed and applied as `factor_`.
        - The recursion builds up results by combining the even and odd parts 
        until the entire DFT of size N is constructed.
        - A bottom-up iterative approach is chosen over a top-down approach for
          performance, but the idea is the same.
    """

    N_min = 16
    N = input.shape[axis]
    
    # use bit-wose operator to check for valid size
    if N & (N - 1) != 0:
        raise NotImplementedError("size of input must be a power of 2 for original Cooley–Tukey")
    elif N >= 2 ** 18:
        raise NotImplementedError("size of input larger than pre-computed twiddle factor range")
    
    # if input length is less than 16, just uses normal DFT
    if N <= N_min:
        return _dft(input, axis, forward)

    # move axis of interest to the last position
    f = numpy.moveaxis(input, axis, -1)
    
    # retrieve correct DFT16 matrix based on whether computation is forward or inverse
    DFT16 = DFT_16 if forward else numpy.conjugate(DFT_16)
    
    # transform data in to blocks of length 16 and apply DFT16
    F = DFT16 @ f.reshape((*f.shape[:-1], N_min, -1))
    
    # reconstruct final results by combining results from problem of smaller size
    while F.shape[-2] < N:
        F_even = F[..., :F.shape[-1] // 2]
        F_odd = F[..., F.shape[-1] // 2:]
        
        twf = TWIDDLE_FACTOR[2 * F.shape[-2]][..., None]
        factor = twf if forward else numpy.conjugate(twf)
        F_odd_transformed = factor * F_odd
        F = numpy.concatenate((F_even + F_odd_transformed, F_even - F_odd_transformed), axis=-2)
    
    result = numpy.moveaxis(F[..., -1], -1, axis)
    return result if forward else result / N


def fft(input: numpy.ndarray, axis: int = -1) -> numpy.ndarray:
    """
    Compute the Fast Fourier Transform (FFT) along a specified axis, which
    converts data from its original domain to the frequency domain.

    Parameters:
    -----------
        input (numpy.ndarray): The input array to be transformed. Can be multi-dimensional.
        axis (int): The axis along which to compute the FFT. Default is -1.

    Returns:
    --------
        numpy.ndarray: The transformed array with the same shape as the input.
    """
    return _fft(input, axis, True)


def ifft(input: numpy.ndarray, axis: int = -1) -> numpy.ndarray:
    """
    Compute the Inverse Fast Fourier Transform (IFFT) along a specified axis, which
    converts data from frequency domain to the origal domain.

    Parameters:
    -----------
        input (numpy.ndarray): The input array to be transformed. Can be multi-dimensional.
        axis (int): The axis along which to compute the IFFT. Default is -1.

    Returns:
    --------
        numpy.ndarray: The transformed array with the same shape as the input.
    """
    return _fft(input, axis, False)

We will test `dft`, `idft`, `fft`, and `ifft` against `numpy.fft.fft` and `numpy.fft.ifft` to compare their accuracy and performance.

In [30]:
def test_fft_and_dft():
    """Test cases for FFT/DFT and IFFT/IDFT"""
    numpy.random.seed(42)
    input_1d = numpy.random.rand(256) + 1j * numpy.random.rand(256)
    input_2d = numpy.random.rand(256, 256) + 1j * numpy.random.rand(256, 256)

    # Test 1D FFT and DFT
    fft_result = fft(input_1d)
    numpy_fft_result = numpy.fft.fft(input_1d)
    assert_allclose(fft_result, numpy_fft_result, err_msg="1D FFT test failed")

    dft_result = dft(input_1d)
    assert_allclose(dft_result, numpy_fft_result, err_msg="1D DFT test failed")

    # Test 1D IFFT and IDFT
    ifft_result = ifft(fft_result)
    numpy_ifft_result = numpy.fft.ifft(numpy_fft_result)
    assert_allclose(ifft_result, numpy_ifft_result, err_msg="1D IFFT test failed")

    idft_result = idft(dft_result)
    assert_allclose(idft_result, numpy_ifft_result, err_msg="1D IDFT test failed")

    # Test 2D input on specific axis
    fft_result_axis_1 = fft(input_2d, axis=1)
    numpy_fft_result_axis_1 = numpy.fft.fft(input_2d, axis=1)
    assert_allclose(fft_result_axis_1, numpy_fft_result_axis_1, err_msg="2D FFT axis=1 test failed")

    dft_result_axis_0 = dft(input_2d, axis=0)
    numpy_dft_result_axis_0 = numpy.fft.fft(input_2d, axis=0)
    assert_allclose(dft_result_axis_0, numpy_dft_result_axis_0, err_msg="2D DFT axis=0 test failed")

    # Test 2D input on specific axis
    ifft_result_axis_1 = ifft(fft_result_axis_1, axis=1)
    numpy_ifft_result_axis_1 = numpy.fft.ifft(numpy_fft_result_axis_1, axis=1)
    assert_allclose(ifft_result_axis_1, numpy_ifft_result_axis_1, err_msg="2D IFFT axis=1 test failed")

    idft_result_axis_0 = idft(dft_result_axis_0, axis=0)
    numpy_idft_result_axis_0 = numpy.fft.ifft(numpy_dft_result_axis_0, axis=0)
    assert_allclose(idft_result_axis_0, numpy_idft_result_axis_0, err_msg="2D IDFT axis=0 test failed")

    print("All tests passed for FFT/DFT and IFFT/IDFT.")

    print("\nTiming comparisons:\n")
    print("Custom dft: ", end="")
    %timeit dft(input_1d)
    print("Custom fft: ", end="")
    %timeit fft(input_1d)
    print("NumPy fft: ", end="")
    %timeit numpy.fft.fft(input_1d)
    print("")
    print("Custom idft: ", end="")
    %timeit idft(input_1d)
    print("Custom ifft: ", end="")
    %timeit ifft(input_1d)
    print("NumPy ifft: ", end="")
    %timeit numpy.fft.ifft(input_1d)

test_fft_and_dft()

All tests passed for FFT/DFT and IFFT/IDFT.

Timing comparisons:

Custom dft: 797 μs ± 38.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Custom fft: 18.4 μs ± 857 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
NumPy fft: 1.46 μs ± 23.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)

Custom idft: 770 μs ± 11.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Custom ifft: 21.3 μs ± 667 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
NumPy ifft: 1.58 μs ± 13.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Now we implement 2D FFT and its inverse, which will be used in our solver because we are solving the equations on a 2D space. We uses the separability property of 2D Fourier transform and combine it with our 1D FFT and IFFT above.

In [32]:
def fft2(input: numpy.ndarray, axes: tuple[int] = (-1, -2)):
    """
    Compute the Fast Fourier Transform (FFT) along two specified axes, which
    converts data from its original domain to the frequency domain.

    Parameters:
    -----------
        input (numpy.ndarray): The input array to be transformed. Can be multi-dimensional.
        axis (tuple[int]): The axes along which to compute the 2D-FFT. Default is (-1, -2).

    Returns:
    --------
        numpy.ndarray: The transformed array with the same shape as the input.
    
    
    """
    for a in axes:
        input = _fft(input, a, True)
    return input


def ifft2(input: numpy.ndarray, axes: tuple[int] = (-1, -2)):
    """
    Compute the Inverse Fast Fourier Transform (IFFT) along two specified axes, which
    converts data from frequency domain to the origal domain.

    Parameters:
    -----------
        input (numpy.ndarray): The input array to be transformed. Can be multi-dimensional.
        axis (tuple[int]): The axes along which to compute the 2D-IFFT. Default is (-1, -2).

    Returns:
    --------
        numpy.ndarray: The transformed array with the same shape as the input.
    """
    for a in axes:
        input = _fft(input, a, False)
    return input 

We test `fft2` and `ifft2` against `numpy.fft.fft2` and `numpy.fft.ifft2` for accuracy and performance.

In [34]:
def test_fft2():
    """Test cases for 2D FFT and IFFT"""
    numpy.random.seed(42)
    input_2d = numpy.random.rand(256, 256) + 1j * numpy.random.rand(256, 256)

    # FFT2
    fft2_result = fft2(input_2d)
    numpy_fft2_result = numpy.fft.fft2(input_2d)
    assert_allclose(fft2_result, numpy_fft2_result, err_msg="2D FFT2 test failed")

    # FFT2 along different axes
    fft2_result_axes_0_1 = fft2(input_2d, axes=(0, 1))
    numpy_fft2_result_axes_0_1 = numpy.fft.fft2(input_2d, axes=(0, 1))
    assert_allclose(fft2_result_axes_0_1, numpy_fft2_result_axes_0_1, err_msg="2D FFT2 axes=(0, 1) test failed")

    # IFFT2
    ifft2_result = ifft2(fft2_result, axes=(-1, -2))
    numpy_ifft2_result = numpy.fft.ifft2(numpy_fft2_result)
    assert_allclose(ifft2_result, numpy_ifft2_result, err_msg="2D IFFT2 test failed")

    ifft2_result_axes_0_1 = ifft2(fft2_result_axes_0_1, axes=(0, 1))
    numpy_ifft2_result_axes_0_1 = numpy.fft.ifft2(numpy_fft2_result_axes_0_1)
    assert_allclose(ifft2_result_axes_0_1, numpy_ifft2_result_axes_0_1, err_msg="2D IFFT2 axes=(0, 1) test failed")

    print("All tests passed for FFT2 and IFFT2.")
    
    print("\nTiming comparisons:\n")
    print("Custom fft2: ", end="")
    %timeit fft2(input_2d)
    print("NumPy fft2: ", end="")
    %timeit numpy.fft.fft2(input_2d)
    print("")
    print("Custom ifft2: ", end="")
    %timeit ifft2(input_2d)
    print("NumPy ifft2: ", end="")
    %timeit numpy.fft.ifft2(input_2d)

test_fft2()

All tests passed for FFT2 and IFFT2.

Timing comparisons:

Custom fft2: 2.54 ms ± 8.55 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
NumPy fft2: 346 μs ± 193 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Custom ifft2: 2.71 ms ± 95.6 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
NumPy ifft2: 382 μs ± 5.31 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### Solver Implementation

**Apply Forces**

The fluid velocity field is influenced by external forces, and in this simplified model, we apply a force field in the x-direction that depends on time and position. The function below updates the velocity filed at the previous time step based on such a force, which only applies in the first second.

In [37]:
def apply_forces(iteration, velocity_x_prev, force_x, TIME_STEP_LENGTH):
    """
    Applies external forces to the velocity field.
    The forcing is time-dependent: it decreases linearly with time until it vanishes when time_current >= 1.

    iteration: current iteration step
    velocity_x_prev: array of previous velocity field in x-direction
    force_x: array of the external force in x-direction
    """
    time_current = iteration * TIME_STEP_LENGTH
    pre_factor = max(1 - time_current, 0.0)
    velocity_x_prev[:] = velocity_x_prev + TIME_STEP_LENGTH * pre_factor * force_x

**Self-Advection by Backtracing and Interpolation**

Here we models the fluid’s own movement advecting itself by backtracking and then interpolating velocity at the backtraced positions.

In [39]:
def backtrace(backtraced_positions, original_positions, direction, TIME_STEP_LENGTH):
    """
    Perform an Euler step backward in time for self-advection and enforce periodic boundary conditions.
    """
    # Euler step backward and wrap positions into [0.0, 1.0)
    backtraced_positions[:] = numpy.mod(
        original_positions - TIME_STEP_LENGTH * direction,
        1.0,
    )

def interpolate_positions(field_interpolated, field, interval_x, interval_y, query_points_x, query_points_y):
    """
    Interpolate the field at the backtraced positions using linear interpolation.
    """
    # Create a regular grid interpolator
    interpolator = RegularGridInterpolator(
        (interval_x, interval_y),
        field,
        method='linear',
        bounds_error=False,
        fill_value=None
    )
    # Stack the query points and evaluate the interpolator
    points = numpy.stack((query_points_x.flatten(), query_points_y.flatten()), axis=-1)
    field_interpolated[:] = interpolator(points).reshape(field_interpolated.shape)

def self_advection_step(velocity_x, velocity_y, 
                        velocity_x_prev, velocity_y_prev, 
                        coordinates_x, coordinates_y, 
                        x_interval, y_interval,
                        backtraced_coordinates_x, backtraced_coordinates_y, 
                        TIME_STEP_LENGTH):
    """
    Performs the self-advection step of the fluid:
    1. Backtrace the grid to find where fluid parcels came from.
    2. Interpolate the velocity field from the previous timestep at these backtraced positions.
    """

    # Backtrace in x and y directions
    backtrace(backtraced_coordinates_x, coordinates_x, velocity_x_prev, TIME_STEP_LENGTH)
    backtrace(backtraced_coordinates_y, coordinates_y, velocity_y_prev, TIME_STEP_LENGTH)

    # Interpolate velocities at the backtraced positions
    interpolate_positions(velocity_x, velocity_x_prev, x_interval, y_interval, backtraced_coordinates_x, backtraced_coordinates_y)
    interpolate_positions(velocity_y, velocity_y_prev, x_interval, y_interval, backtraced_coordinates_x, backtraced_coordinates_y)


**Diffusion (In Fourier Domain)**

After FFT is applied, the velocity is updated with the implicit method introduced before.


In [41]:
def diffusion_step(velocity_x_fft, velocity_y_fft, wavenumbers_norm, TIME_STEP_LENGTH, KINEMATIC_VISCOSITY):
    """
    Performs the diffusion step of the fluid.
    """
    velocity_x_fft[:] = velocity_x_fft / (1 + TIME_STEP_LENGTH * KINEMATIC_VISCOSITY * wavenumbers_norm**2)
    velocity_y_fft[:] = velocity_y_fft / (1 + TIME_STEP_LENGTH * KINEMATIC_VISCOSITY * wavenumbers_norm**2)

**Projection (In Fourier Domain)**

Here we project the velocity field onto a divergence-free (incompressible) field. A pseudo-pressure is introduced as an intermediate step. 

In [43]:
def projection_step(velocity_x_fft, velocity_y_fft, 
                   normalized_wavenumbers_x, normalized_wavenumbers_y):
    """
    Performs the Projection step of the fluid.
    """
    
    # Compute pseudo-pressure in Fourier domain
    pressure_fft = (
        velocity_x_fft * normalized_wavenumbers_x +
        velocity_y_fft * normalized_wavenumbers_y
    )

    # Project velocities to be incompressible
    velocity_x_fft[:] = velocity_x_fft - pressure_fft * normalized_wavenumbers_x
    velocity_y_fft[:] = velocity_y_fft - pressure_fft * normalized_wavenumbers_y

**Particles Advection**

Utility function of interpolating velocities at particle locations and updating particle locations is deinfed here.

In [45]:
def interpolate_for_particles(field, interval_x, interval_y, particle_x, particle_y):
    """
    Interpolate a 2D field at arbitrary particle positions.
    Returns a 1D array corresponding to the field value at each particle.
    """
    interpolator = RegularGridInterpolator(
        (interval_x, interval_y),
        field,
        method='linear',
        bounds_error=False,
        fill_value=None
    )
    points = numpy.stack((particle_x, particle_y), axis=-1)
    return interpolator(points)

def particle_advection(velocity_x, velocity_y,
                       x_interval, y_interval,
                       particle_x, particle_y,
                       TIME_STEP_LENGTH): 
    """
    Tracking particle position for visualization.
    """
   # Interpolate velocity at particle locations
    particle_vel_x = interpolate_for_particles(velocity_x, x_interval, y_interval, particle_x, particle_y)
    particle_vel_y = interpolate_for_particles(velocity_y, x_interval, y_interval, particle_x, particle_y)

    # Update particle positions (Euler forward step)
    particle_x[:] = particle_x + TIME_STEP_LENGTH * particle_vel_x
    particle_y[:] = particle_y + TIME_STEP_LENGTH * particle_vel_y

    # Apply periodic boundary conditions
    particle_x[:] = particle_x % 1.0
    particle_y[:] = particle_y % 1.0

#### Runtime Simulation

In [47]:
# Constants
N_POINTS = 256
KINEMATIC_VISCOSITY = 0.0001
TIME_STEP_LENGTH = 0.007
N_TIME_STEPS = 300
N_PARTICLES = 2000

In [48]:
# Create spatial grid
element_length = 1.0 / (N_POINTS - 1)
x_interval = numpy.linspace(0.0, 1.0, N_POINTS)
y_interval = numpy.linspace(0.0, 1.0, N_POINTS)
coordinates_x, coordinates_y = numpy.meshgrid(x_interval, y_interval, indexing='ij')

# Compute wavenumbers for Fourier transforms
wavenumbers_1d = freq(N_POINTS) * N_POINTS
wavenumbers_x, wavenumbers_y = numpy.meshgrid(wavenumbers_1d, wavenumbers_1d, indexing='ij')
wavenumbers_norm = numpy.sqrt(wavenumbers_x**2 + wavenumbers_y**2)

# Avoid division by zero in normalization
wavenumbers_norm[wavenumbers_norm == 0] = 1.0
normalized_wavenumbers_x = wavenumbers_x / wavenumbers_norm
normalized_wavenumbers_y = wavenumbers_y / wavenumbers_norm

# Define the external force field (two opposing Gaussian forces)
force_x = 100.0 * (
    numpy.exp(- 1.0 / (2 * 0.005) * ((coordinates_x - 0.2)**2 + (coordinates_y - 0.45)**2))
    -
    numpy.exp(- 1.0 / (2 * 0.005) * ((coordinates_x - 0.8)**2 + (coordinates_y - 0.55)**2))
)

# Initialize arrays
backtraced_coordinates_x = numpy.zeros_like(coordinates_x)
backtraced_coordinates_y = numpy.zeros_like(coordinates_y)
velocity_x = numpy.zeros_like(coordinates_x)
velocity_y = numpy.zeros_like(coordinates_y)
velocity_x_prev = numpy.zeros_like(velocity_x)
velocity_y_prev = numpy.zeros_like(velocity_y)

# For FFT operations, use complex data types
velocity_x_fft = numpy.zeros_like(velocity_x, dtype=numpy.complex128)
velocity_y_fft = numpy.zeros_like(velocity_y, dtype=numpy.complex128)
pressure_fft = numpy.zeros_like(coordinates_x, dtype=numpy.complex128)

# Initialize particles
# Let's place a number of particles randomly in the domain
particle_x = numpy.random.rand(N_PARTICLES)
particle_y = numpy.random.rand(N_PARTICLES)

plt.figure(figsize=(8, 6))

# Time-stepping loop
for iter in range(N_TIME_STEPS):
    apply_forces(iter,velocity_x_prev,force_x, TIME_STEP_LENGTH)
    self_advection_step(velocity_x, velocity_y, 
                        velocity_x_prev, velocity_y_prev, 
                        coordinates_x, coordinates_y, 
                        x_interval, y_interval,
                        backtraced_coordinates_x, backtraced_coordinates_y,
                        TIME_STEP_LENGTH)
    # FFT
    velocity_x_fft = fft2(velocity_x)
    velocity_y_fft = fft2(velocity_y)
    
    diffusion_step(velocity_x_fft, velocity_y_fft, wavenumbers_norm, TIME_STEP_LENGTH, KINEMATIC_VISCOSITY)
    projection_step(velocity_x_fft, velocity_y_fft, 
                   normalized_wavenumbers_x, normalized_wavenumbers_y)
    
    # IFFT
    velocity_x = numpy.real(ifft2(velocity_x_fft))
    velocity_y = numpy.real(ifft2(velocity_y_fft))

    # Update previous velocities for the next iteration
    velocity_x_prev[:] = velocity_x
    velocity_y_prev[:] = velocity_y

    particle_advection(velocity_x, velocity_y,
                       x_interval, y_interval,
                       particle_x, particle_y,
                       TIME_STEP_LENGTH)
    
    # Visualization of the vorticity (curl of the velocity field)
    d_u__d_y = numpy.diff(velocity_x, axis=1)[1:, :]
    d_v__d_x = numpy.diff(velocity_y, axis=0)[:, 1:]
    curl = d_u__d_y - d_v__d_x

    plt.clf()
    plt.imshow(
        curl.T,
        origin='lower',
        extent=[0, 1, 0, 1],
        cmap='seismic',
        aspect='equal'
    )
    plt.title(f"Fluid Flow Simulation at Time Step {iter+1}")
    plt.colorbar(label='Vorticity')

    # Overlay the particle positions
    plt.scatter(particle_x, particle_y, s=2, c='black', alpha=0.7)

    plt.pause(0.005)

plt.show()

## Discussion [10 pts]

Upon reviewing our simulations, we believe that our project successfully achieved its goals, which is to deliver a stable and visually correct simulation of fluid dynamics. 

The custom-implemented Fourier Transform functions were rigorously tested against `numpy`'s native functions using `numpy.testing.assert_allclose`, confirming their numerical correctness. Moreover, by carefully following the methodology outlined in Jos Stam's paper, we ensured that our solver maintained the unconditional stability characteristic of Stam's approach with arbitrary time step. 

We think our implementations of the FFT are also effective and successful in terms of their performance. For input size of 256, our `fft` and `ifft` are 60 to 70 times faster than our own `dft` and `idft` functions. Compared to `numpy`'s highly optimized functions, our implementations are 12 - 13 times slower. Given that `numpy.fft` are implemented in C language, I think our implementation in Python is already decent. For our `fft2` and `ifft2`, our implementation is only 7 - 8 times slower. More importantly, when substituting our `fft2` and `ifft2` into the solver, we did not observe any noticeable slowdown or lag. 

From the simulation animations, we think the fluid part also worked. The plots depicting fluid flows exhibit the behaviors we anticipated, providing visually intuitive and sensible outcomes. We also tested different initial conditions for the forces applied, and the results were consistent with our expectations, which further validates the effectiveness of the solver on a computer graphics level. However, it is indeed hard to judge how accurate the result is because, as mentioned above and in Jos Stam's paper, this unconditionally stable solver trade accuracy for stability and efficiency. It is not adequate and should not be used for precise engineering and physics endeavors. Yet, on a visual demonstration level, we think it was able to does its job.

Looking ahead, there are several directions for further development. One involves enhancing the FFT implementation. Currently, our solver relies on the original Cooley-Tukey algorithm, which restricts length of input to powers of two. Incorporating advanced algorithms that leverage prime factorization and different twiddle factors would enable support for arbitrary grid sizes. On the fluid solver side, an interesting next step would be to introduce rigid bodies into the simulation space, which will let us visualize fluid-object interactions, such as the aerodynamics of vehicles. Although this was mentioned initially in the project proposal, time constraints prevented us from implementing it here. Such an addition would also require modifications to the solver to accommodate for non-periodic boundary conditions.

Overall, we think the project not only met its objectives but also gave us a good framework for future improvements in more versatile simulations. We'd love to continue with the implementations on our own. Thank you, Professor Spiegelman, for reading!